In [2]:
# Ignore this cell
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/datasets/finance_social/Dataset')
os.listdir()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


['StockTwits',
 'Twitter',
 'ReutersNews',
 'twiters_label_data_Feb-02-2020.xlsx',
 'Stocktwits Label Model training with twiter data.ipynb',
 'github data',
 'INTENTS and UTTERANCE.yaml',
 'Keras_FNN_model.json',
 'finalized_Keras_model.sav',
 'Stocktwits Label Model training with twiter data - Final.ipynb',
 'Keras KNN test result 1.PNG',
 'Keras KNN test result 1a.PNG',
 'finalized_Keras_model_1.sav',
 'Keras_FNN_model_1.json',
 'Keras_FNN_model.h5',
 'Stocktwits Keras FNN for sentiment analysis-Final.ipynb',
 'finalized_SVM_model.sav',
 'chatbot_training.yaml',
 'stocktwits_sentiment_subjectivity_Feb-01-2020_p.xlsx']

In [3]:
from pathlib import Path
import json
import re
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import html
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from keras.callbacks import ModelCheckpoint,CSVLogger
import spacy
import keras
from keras import models
from keras import layers
from keras import regularizers
print (keras.__version__)
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize, wordpunct_tokenize

Using TensorFlow backend.


2.2.5
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
!python -m spacy download en_core_web_md
import en_core_web_md
nlp = en_core_web_md.load()

     |████████████████████████████████| 95.4MB 1.1MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-cp36-none-any.whl size=97126236 sha256=bade8f400ceb43b3248ffa066d09d6ca843680b017a97765a15a3d1ca58b5221
  Stored in directory: /tmp/pip-ephem-wheel-cache-2pvglljj/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
stocktwits_dir = Path("StockTwits")
twitter_dir = Path("Twitter")
stocktwits_filename = 'stocktwits.pkl'
model_path= '/content/drive/My Drive/logs/0.89_96_64_32_2/weights-improvement-289-0.89.hdf5'

In [0]:
def preprocess(text):
  processed = re.sub(r'\B\$\s?\w+(\.\w+)?\b\s*', '', text)
  processed = re.sub(r',\s+','', processed) # commas not informative for sentiment
  processed = re.sub(r'\s', ' ', processed) # replace new line and tabspace with single space
  processed = re.sub('http\S+', '',  processed) # remove URLs
  processed = html.unescape(processed) # unescape html text
  return processed.strip()

In [0]:
def extract_features(data, txtcol):
  data[txtcol] = data[txtcol].apply(preprocess)
  ul_docs = data[txtcol].apply(lambda x: nlp(x))
  ul_X = ul_docs.apply(lambda x: x.vector)
  ul_X = np.array(ul_X.tolist())
  return ul_X

In [0]:
def label_sentiments(data, txtcol, model, feature_file=None):
  ul_X = np.load(feature_file) if feature_file else extract_features(data, txtcol)
  ul_y = model.predict(ul_X)
  ul_labels = np.argmax(ul_y,axis=1)
  labels= pd.Series(ul_labels).replace({0:'Bearish',1:'Bullish'})
  data['bearish_score'] = ul_y[:,0]
  data['bullish_score'] = ul_y[:,1]
  data['sentiment_pred']= labels
  return data, ul_X, ul_y

In [0]:
model_path= '/content/drive/My Drive/logs/0.89_96_64_32_2/weights-improvement-289-0.89.hdf5'
model = models.load_model(model_path)
# model.summary()

# Predictions on unseen Twitter Data

In [7]:
twtr_sheets = pd.read_excel(twitter_dir/'stock_tweets_Mar-02-2020.xlsx', sheet_name=None)
twtr_data = pd.concat(list(twtr_sheets.values()))
twtr_data.columns

Index(['Date', 'Text', 'favorite_count', 'retweet_count', 'ticker'], dtype='object')

In [24]:
twtr_lbld,_,_ = label_sentiments(twtr_data, 'Text', model, feature_file='twitter_ul.npy')
twtr_lbld.to_pickle(twitter_dir / 'twitter_labelled_v1.pkl')
twtr_lbld['sentiment_pred'].value_counts()

Bullish    64772
Bearish    28341
Name: sentiment_pred, dtype: int64